In [0]:
from google.colab import files

# Install Kaggle library
!pip install -q kaggle
uploaded = files.upload()

In [0]:
!chmod 770 /content/kaggle.json

In [0]:
!mkdir /root/.kaggle/
!cp /content/kaggle.json /root/.kaggle/kaggle.json

In [0]:
from google.colab import drive 
drive.mount('/content/drive')

In [0]:
!kaggle competitions download -c ml2019spring-hw7

In [0]:
!unzip images.zip
!unzip test_case.csv.zip
!unzip visualization.npy.zip


In [0]:
!tar -zxvf ./Aberdeen.tar.tgz

In [0]:
import os
import sys
import numpy as np 
from skimage.io import imread, imsave 
from skimage import transform
import matplotlib.pyplot as plt


In [0]:
def load_data(PAYH) ->  list :
    filelist =[f for f in os.listdir(PAYH) if not f.startswith('.') ]  
    # Record the shape of images
    img_shape = imread(os.path.join(PAYH,filelist[0])).shape 

    img_data = []
    for filename in filelist:
        tmp = imread(os.path.join(PATH,filename))  
        img_data.append(transform.resize(tmp, (128,128,3) ) .flatten())
    return img_data 
def process(M): 
    M -= np.min(M)
    M /= np.max(M)
    M = (M * 255).astype(np.uint8)
    return M

# Calculate mean & Normalize
class PCA(object):
    def __init__(self,  compnent):
        self.compnent = compnent
    def fit(self , train_data):
        self.mean = np.mean(train_data, axis = 0)  
        train_data -= self.mean 
        # Use SVD to find the eigenvectors not use np.linalg.eig , cause not square matriz 
        self.u, self.s, v = np.linalg.svd(training_data.T, full_matrices = False)  
        self.u=self.u[:,:self.compnent]
        self.s=self.s[:self.compnent]
        return 
    def infer(self,test_data ):
        test = test_data- self.mean
        return np.dot( np.dot(test , self.u) ,self.u.T) 

        


In [0]:

PATH = './Aberdeen'   
img_data = load_data(PATH)
training_data = np.array(img_data).astype('float32')
pca = PCA(compnent = 5 )
pca.fit(training_data)
test_image = ['1.jpg','10.jpg','22.jpg','37.jpg','72.jpg'] 
test_image_path =[os.path.join(PATH,t) for t in test_image ]
test_data =list()
test_data.extend([transform.resize(imread(file), (128,128,3) ).flatten()  for file in test_image_path ])
test_data = np.array(test_data).astype('float32')
test_result = pca.infer(test_data) 
###  show()
plt.imshow( test_data[0].reshape(128,128,3)) 
plt.show()
plt.imshow(process (test_result)[0].reshape(128,128,3))
plt.show()

In [0]:
!nvidia-smi

In [0]:
# standard library
import argparse
import csv
import time
import sys
import os
# other library
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
# PyTorch library
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data 
from torch.autograd import Variable

In [0]:
# preprocess
class Dataset(data.Dataset):
    def __init__(self, imgae_dir):
        self.total_img = []
        for i in range(1, 40001):
            print("loading image %d/40000" % i, end='\r')
            fname = os.path.join(imgae_dir, "%06d.jpg" % (i))
            img = Image.open(fname)
            img.load()
            row = np.asarray(img)
            self.total_img.append(row)
            
        # since at pytorch conv layer, input=(N, C, H, W)
        self.total_img = np.transpose(np.array(self.total_img, dtype=float), (0, 3, 1, 2))
        # normalize
        self.total_img = (self.total_img ) / 255.0
        #np.random.shuffle(self.total_img)
        self.train_img = self.total_img[:35501]
        self.val_img = self.total_img[35501:]
        print("=== total image shape:",  self.total_img.shape)
        # shape = (40000, 3, 32, 32)

    def __len__(self):
        return len(self.total_img)

    def __getitem__(self, index):
        return(self.total_img[index])
def process(M): 
    M -= np.min(M)
    M /= np.max(M)
    M = (M * 255).astype(np.uint8)
    return M
    
imgae_dir ='./images/'
dataset = Dataset(imgae_dir)


In [0]:
# Model Construction
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def guassian_noise(input ,device ,mean = 0 ,stddev = 0.01 ):
    return torch.clamp (input + Variable(torch.randn(input.size()).to(device) * stddev),0,1   )
class AutoEncoder(nn.Module):
    def __init__(self, image_shape, latent_dim):
        super(AutoEncoder, self).__init__()
        self.shape = image_shape
        self.latent_dim = latent_dim
        # CNN  output_shape = (image_shape-kernel_size+2*padding)/stride + 1
        self.encoder = nn.Sequential(  # batch ,3 , N ,N
            nn.Conv2d(3, 16, 3, padding=1 ,stride = 1 ),  #batch , 16 ,  32, 32
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),   # batch , 16 , 16,16

            nn.Conv2d(16, 64, 3, padding=1,stride =1 ),  # batch , 64 , 16,16
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),   # batch , 16 ,8,8

            nn.Conv2d(64, 256, 3, padding=1,stride =1 ),  # batch , 64 , 8,8
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2),    # batch , 256 , 4,4
        )
        # assume output shape is (Batch, channel, 1, 1)
        
        self.fc1 = nn.Linear(256*4*4 , self.latent_dim)
        self.fc2 = nn.Linear(self.latent_dim, 256*4*4)
        #self.fc3 = nn.Linear(self.h_dim , self.latent_dim)
        # ConvTranspose2d output_shape = (input_shape-1)*stride + output_padding -2*padding +kernel_size
        self.decoder = nn.Sequential(
           # TODO: define your own structure
           nn.ConvTranspose2d(256*4*4, 256 ,3 , stride=1 ),   # batch , 64 ,3,3
           nn.ReLU(),
           nn.BatchNorm2d(256),
           nn.Upsample(scale_factor=2, mode='bilinear',align_corners=False) , #batch,64,6,6
           
           nn.ConvTranspose2d(256 ,64 ,2 ,stride=1 ) ,    # batch , 64, 7,7
           nn.ReLU(),
           nn.BatchNorm2d(64),
           nn.Upsample(scale_factor=2, mode='bilinear' ,align_corners=False) ,#batch,64,14,14
           

           nn.ConvTranspose2d(64 ,16 ,2 ,stride=1 ) , #batch ,16 , 15,15
           nn.ReLU(), 
           nn.BatchNorm2d(16),
           nn.Upsample(scale_factor=2, mode='bilinear' ,align_corners=False) , #batch,64,30,30
           
           nn.ConvTranspose2d(16 ,3 ,3 ,stride=1 ) , #batch ,16 , 32,32
           nn.ReLU(), 
        )
   # def reparameterize(self, mu, logvar):
    #    sigma = torch.exp(logvar)
     #   std_z = torch.from_numpy(np.random.normal(0, 1, size=sigma.size())).float()
      #  return mu + sigma * Variable(std_z, requires_grad=False).to(device) 
     
    #def encode(self,x):
      #  h = F.relu(self.fc1(x))
      #  return self.fc2(h) , self.fc3(h)
    def forward(self, x):
        x = self.encoder(x)
        # flatten
        x = x.view(len(x), -1) # batch , (channel*pixel*pixel)
        
        #mu , log_var = self.encode(x)
        #encoded = self.reparameterize( mu ,log_var )

        encoded = self.fc1(x)
        x = F.relu(self.fc2(encoded))  

        #encoded = self.fc2(x)
        #x = F.relu(self.fc2(encoded))  
        x = x.view(-1, 256*4*4, 1, 1) 
        x = self.decoder(x)
        return encoded, x #,mu, log_var

train = data.DataLoader(dataset, batch_size=256, shuffle=True)
validation = data.DataLoader(dataset.val_img, batch_size=500, shuffle=False)
total_data = data.DataLoader(dataset, batch_size=256, shuffle=False)



In [0]:
device

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
model = AutoEncoder(image_shape=dataset.__getitem__(0).shape , latent_dim = 128 )
model.to(device)
total = sum(p.numel() for p in model.parameters())
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, 
                             weight_decay=1e-5)
                             
num_epochs=60
model_path = '/content/drive/My'' ''Drive/Colab_model/'
if os.path.isfile(model_path+'autoencoder_model'):
    #model.load_state_dict(torch.load(model_path+'autoencoder_model')) #for gpu 
    model.load_state_dict(torch.load(model_path+'autoencoder_model' , map_location=torch.device('cpu')))

best_loss =100
val_loss=[]
train_loss =[]
for epoch in range(num_epochs):
    epoch_start_time = time.time()
    total_loss  = 0
    model.train()
    for idx,image in enumerate(train):
        image = image.to(device , dtype=torch.float)
        image_noise = guassian_noise(image , device)
        _, reconsturct = model(image_noise)

        #kl_div = 0.5 * torch.mean( 1- mu.pow(2) - log_var.exp() + log_var )
        loss = criterion(reconsturct, image) #- kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += (loss.item() / len(train))
        print(loss.item(),end='\r')
    print ('[%03d/%03d] %2.2f sec(s) ' % (epoch+1, num_epochs, \
                (time.time() - epoch_start_time) ))
    print("\n Training | Loss :%.4f " % total_loss)
    train_loss.append(total_loss)
    # validation set
    model.eval()
    total_loss = 0
 
    with torch.no_grad():
        for idx, image in enumerate(validation):
                image = image.to(device , dtype=torch.float)
                image_noise = guassian_noise(image , device)
                _, reconstruct = model(image_noise)
               # kl_div = 0.5 * torch.mean(1- mu.pow(2) - log_var.exp() + log_var )
                loss = criterion(reconstruct, image)# - kl_div
                total_loss += (loss.item() / len(validation))
        print("[%03d/%03d]%2.2f sec Validation | Loss:%.4f " \
                % (epoch+1, num_epochs,(time.time() - epoch_start_time), total_loss) )
        val_loss.append(total_loss)
        if(best_loss > total_loss):
            best_loss = total_loss
            print("saving model with val loss %.4f...\n" % total_loss)
            torch.save(model.state_dict(),model_path+'autoencoder_model')
            print('Model Saved!!! ')
    if(epoch%5==0):
        
        index = 20
        sh =  np.transpose(dataset.__getitem__(index), (  1,2, 0) )
        plt.imshow(sh)
        plt.show()
        model.eval()
        _ , reconsturct = model(\
        torch.FloatTensor( np.expand_dims( dataset.__getitem__(index),axis=0 ) ).cuda()  )


        sh =  np.transpose(process ( reconsturct[0].cpu().data.numpy()), (  1,2, 0) )
        plt.imshow(sh)
        plt.show()
        
plt.plot(range(num_epochs),train_loss)
plt.plot(range(num_epochs),val_loss)
plt.show()


In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 
model = AutoEncoder(image_shape=dataset.__getitem__(0).shape , latent_dim = 128 )
model.to(device)
model_path = '/content/drive/My'' ''Drive/Colab_model/'
if os.path.isfile(model_path+'autoencoder_model'):
    model.load_state_dict(torch.load(model_path+'autoencoder_model'))

def clustering(model, device, loader,n_iter, reduced_dim):
    model.eval()
    da = list()
    #latent_vec = torch.tensor([]).to(device, dtype=torch.float)
    for idx, image in enumerate(loader):
        print("predict %d / %d" % (idx, len(loader)) , end='\r')
        image = image.to(device, dtype=torch.float)
        latent, _ = model(image)
        #latent_vec = torch.cat((latent_vec, latent), dim=0)
        latent = latent.cpu().detach().numpy()
        da.extend(latent)
    #print(latent_vec.shape)
    # shape = (40000, latent_dim)
    # n_components < 4
    #tsne = TSNE(n_components=reduced_dim, verbose=1, perplexity=50, n_iter=n_iter)
    #latent_vec = tsne.fit_transform(np.array(da))

    pca = PCA(n_components=reduced_dim, copy=False, whiten=True, svd_solver='full')
    latent_vec = pca.fit_transform(np.array(da))

    kmeans = KMeans(n_clusters=2, random_state=0, max_iter=n_iter).fit(latent_vec)
    return kmeans.labels_
def read_test_case(path):
    dm = pd.read_csv(path)
    img1 = dm['image1_name']
    img2 = dm['image2_name']
    test_case = np.transpose(np.array([img1, img2]))
    return test_case
def prediction(label, test_case, output):
    result = []
    for i in range(len(test_case)):
        index1, index2 = int(test_case[i][0])-1, int(test_case[i][1])-1
        if label[index1] != label[index2]:
            result.append(0)
        else:
            result.append(1)
    
    result = np.array(result)
    with open(output, 'w') as f:
        f.write("id,label\n")
        for i in range(len(test_case)):
            f.write("%d,%d\n" % (i, result[i]))


In [0]:
label = clustering( model ,device , total_data,n_iter=300 ,reduced_dim=128 )
test_case = read_test_case('./test_case.csv')
prediction(label , test_case , './submit.csv')


In [0]:
test_set = np.load('./visualization.npy')/255.0
test_set = np.transpose(np.array(test_set, dtype=float), (0, 3, 1, 2))
test = data.DataLoader(test_set, batch_size=10, shuffle=False)

model.eval()
model.to(device , dtype=torch.float)
criterion = nn.MSELoss()
test_loss=0
for t in (test):
    t = t.to(device , dtype=torch.float)
    _ , img = model(t)
    loss = criterion(img,t)
    test_loss +=loss
print(test_loss/len(test))
    